In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel('./file/4.part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：337


['81722',
 '81723',
 '81724',
 '81725',
 '81726',
 '81727',
 '81728',
 '81729',
 '81730',
 '81731',
 '81732',
 '81733',
 '81734',
 '81735',
 '81736',
 '81737',
 '81738',
 '81740',
 '81741',
 '81742',
 '81743',
 '81744',
 '81746',
 '81747',
 '81748',
 '81749',
 '81750',
 '81751',
 '81752',
 '81753',
 '81754',
 '81755',
 '81756',
 '81757',
 '81758',
 '81759',
 '81760',
 '81761',
 '81763',
 '81765',
 '81766',
 '81767',
 '81768',
 '81769',
 '81770',
 '81771',
 '81772',
 '81773',
 '81774',
 '81775',
 '81777',
 '81779',
 '81781',
 '81783',
 '81785',
 '81787',
 '81789',
 'CAM1A',
 'CC81762',
 'CC81764',
 'CK620486',
 'CK620487',
 'CK620634',
 'CK620888',
 'CK620889',
 'CK621245',
 'CK621246',
 'CK621264',
 'CK621265',
 'CK621266',
 'CK621267',
 'CK621355',
 'CK621356',
 'CK621474',
 'CK621475',
 'CK621897',
 'CK622033',
 'CK622034',
 'CK622735',
 'CK622736',
 'CK622856',
 'CK622857',
 'CK622993',
 'CK622995',
 'CK623014',
 'CK623015',
 'CK623022',
 'CK623023',
 'CK623125',
 'CK623126',
 'CK62

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(f'''https://www.drivparts.com/driv/partfinder/api.catalog.product?brand=corporate&locale=en_US&country_code=US&part_number={input_.loc[a, 'Part Number']}&brand_code={input_.loc[a, 'Brand']}''',
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            json_ = resp.json()
            
            if json_:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [6]:
list_key = ['gtin_qualifier_name',
            'quantity_size_uom',
            'part_attributes',
            'population_status',
            'title',
            'descriptions',
            'default_application_qty',
            'minimum_order_quantity_uom',
            'brand',
            'hybris_brand_code',
            'quantity_per_application',
            'gtin',
            'item_id',
            'universal_part',
            'minimum_order_quantity',
            'gtin_qualifier_code',
            'part_type',
            'product_category',
            'brand_name',
            'quantity_size',
            'quantity_per_application_qualifier',
            'extended_informations',
            'container_type',
            'part_number',
            'quantity_per_application_uom',
            'wtb_brand_name',
            'dam_assets']

for key in json_:
    if key not in list_key:
        print('fuck')

In [7]:
dict_specification, list_attribute = {}, json_['part_attributes'] if 'part_attributes' in json_ else []
for i, attribute in zip(range(len(list_attribute)), list_attribute):
    dict_specification[str(i)] = {attribute['attribute'].strip(): f'''{attribute['attribute_value'].strip()} {attribute['attribute_uom_label'].strip()}''' if 'attribute_uom_label' in attribute else attribute['attribute_value'].strip()}

dict_specification

{}

In [8]:
dict_src, list_src = {}, json_['dam_assets']['productPrimaries']
for i, src in zip(range(len(list_src)), list_src):
    dict_src[str(i)] = f'''https://www.drivparts.com{src['url'].strip()}'''

dict_src

{'0': 'https://www.drivparts.com/bin/fmmp/1706912060618.fmmp.cache-buster/content/dam/sync/NA/assets/moog/1706762110901/I-NA-BCCH-1969/jcr:content/renditions/cq5dam.thumbnail.319.319.png',
 '1': 'https://www.drivparts.com/bin/fmmp/1706912061364.fmmp.cache-buster/content/dam/sync/NA/assets/moog/1706762110901/I-NA-BCCH-1970/jcr:content/renditions/cq5dam.thumbnail.319.319.png'}

In [9]:
import json

df_temp = pd.DataFrame([{'No': '',
                         'Type': json_['part_type'].strip(),
                         'Part Number': input_.loc[a, 'Part Number'],
                         'Brand': input_.loc[a, 'Brand'],
                         'Brand Name': json_['brand_name'].strip(),
                         'GTIN': json_['gtin'].strip(),
                         'Url': input_.loc[a, 'Url'],
                         'Json_Src': json.dumps(dict_src)},
                         'Json_Specification': json.dumps(dict_specification)])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Type,Part Number,Brand,Brand Name,GTIN,Json_Specification,Url,Json_Src
0,,Coil Spring Set,81722,BCCH,MOOG Chassis Products,00080066703060,{},https://www.drivparts.com/part-details.html?br...,"{""0"": ""https://www.drivparts.com/bin/fmmp/1706..."


In [10]:
crawler_status = 'ok'

crawler_status

'ok'

In [11]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [12]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Type', 'Part Number'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct['No'] = [i+1 for i in range(len(output_correct))]
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,No,Type,Part Number,Brand,Brand Name,GTIN,Json_Specification,Url,Json_Src
0,1,Coil Spring Set,81722,BCCH,MOOG Chassis Products,00080066703060,{},https://www.drivparts.com/part-details.html?br...,"{""0"": ""https://www.drivparts.com/bin/fmmp/1706..."


In [13]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
